In [1]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [2]:
#Load data set

from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
dataset[2]

{'text': '### Human: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant: Sure! Let\'s say you want to build a model which can distinguish between images of cats and dogs. You gather your dataset, consisting of many cat and dog pictures. Then you put them through a neural net of your choice, which produces some representation for each image, a sequence of numbers like [0.123, 0.045, 0.334, ...]. The problem is, if your model is unfamiliar with cat and dog images, these representations will be quite random. At one time a cat and a dog picture could have very similar representations (their numbers would be close to each other), while at others two cat images may be represented far apart. In simple terms, the model wouldn\'t be able to tell cats and dogs apart. This is where contrastive learning comes in.\n\nThe point of contrastive learning is to take pairs of samples (in this case images of cats and dogs), then train 

In [4]:
import pandas as pd
import json

# Read data from the JSON file into a list of dictionaries

with open("../convLLM/generateQApairs/pairs.json", "r") as json_file:
    data = json.load(json_file)
for item in data:
    del item['reference_text']

In [5]:
data

[{'question': 'What is the purpose of Flux Balance Analysis (FBA) models?',
  'answer': 'The purpose of Flux Balance Analysis (FBA) models is to predict the metabolic behavior of an organism by quantifying the flow of metabolites through its metabolic network. FBA models are used to study and understand cellular metabolism, particularly in the context of predicting the growth rate or biomass yield of an organism under different conditions. By accounting for the stoichiometry of reactions and the constraints imposed by the availability of nutrients, FBA models can simulate the distribution of fluxes through metabolic pathways and identify optimal metabolic states that maximize the production of desired metabolites or biomass. FBA models are valuable tools in systems biology and can be used to guide metabolic engineering efforts, optimize bioprocesses, and explore the metabolic capabilities of organisms.'},
 {'question': 'What file formats can be used to upload FBA models into KBase?',
 

In [6]:
df = pd.DataFrame(data)
df.head()

,question,answer
0,What is the purpose of Flux Balance Analysis (...,The purpose of Flux Balance Analysis (FBA) mod...
1,What file formats can be used to upload FBA mo...,Flux Balance Analysis (FBA) models can be uplo...
2,How many files are required when uploading an ...,"When uploading an FBA model in the TSV format,..."
3,Where can additional SBML FBA Models for vario...,Additional SBML FBA Models for various organis...
4,What is the recommended file format for creati...,The recommended file format for creating a COB...


In [7]:
print(df['question'][0],df['answer'][0])

What is the purpose of Flux Balance Analysis (FBA) models? The purpose of Flux Balance Analysis (FBA) models is to predict the metabolic behavior of an organism by quantifying the flow of metabolites through its metabolic network. FBA models are used to study and understand cellular metabolism, particularly in the context of predicting the growth rate or biomass yield of an organism under different conditions. By accounting for the stoichiometry of reactions and the constraints imposed by the availability of nutrients, FBA models can simulate the distribution of fluxes through metabolic pathways and identify optimal metabolic states that maximize the production of desired metabolites or biomass. FBA models are valuable tools in systems biology and can be used to guide metabolic engineering efforts, optimize bioprocesses, and explore the metabolic capabilities of organisms.


In [8]:
# Save the data to a JSON file
with open("data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("JSON data has been saved to output.json")

JSON data has been saved to output.json


In [9]:
data = load_dataset("json",data_files="data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
data["train"][1]

{'question': 'What file formats can be used to upload FBA models into KBase?',
 'answer': 'Flux Balance Analysis (FBA) models can be uploaded into KBase using three different file formats. These formats include:\n\n1. Systems Biology Markup Language (SBML): FBA models can be uploaded as SBML files with either the .sbml or .xml file extension. SBML is a standard format for representing computational models in systems biology.\n\n2. Excel: FBA models can also be uploaded as Excel files with the .xls extension. This allows users to create and edit FBA models using spreadsheet software.\n\n3. Tab-separated values (TSV): FBA models can be uploaded as TSV files with the .tsv extension. When using the TSV format, there will be two separate files corresponding to chemical compounds and reactions.\n\nIt is important to note that when uploading an FBA model in the TSV format, the model should be formatted correctly with the appropriate columns and data.\n\nIn summary, FBA models can be uploaded 

In [11]:
def generate_prompt(data_point):
    return f"""
    <human>: {data_point['question']}
    <assistant>: {data_point['answer']}
    """.strip()

In [12]:
model_name = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [14]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt,padding=True,truncation=True)
    return tokenized_full_prompt

In [15]:
data = data['train'].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [36]:
data

Dataset({
    features: ['question', 'answer', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 18
})

In [23]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [28]:
from peft import LoraConfig, get_peft_model


lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 130547712 || all params: 3739292544 || trainable%: 3.4912409356543783


In [29]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "cosine"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    #group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [30]:
from trl import SFTTrainer

max_seq_length = 512

# trainer = SFTTrainer(
#     model=model,
#     train_dataset=data,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     args=training_arguments,
# )
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,mlm=False),
    args=training_arguments,
)

In [31]:
model.config.use_cache = False

In [32]:
trainer.train()

Step,Training Loss
10,2.302200
20,1.395300
30,0.612100
40,0.193000
50,0.096400
60,0.055400
70,0.044900
80,0.042500
90,0.039800
100,0.038900


TrainOutput(global_step=500, training_loss=0.12523856300115585, metrics={'train_runtime': 822.6604, 'train_samples_per_second': 2.431, 'train_steps_per_second': 0.608, 'total_flos': 4559091949294848.0, 'train_loss': 0.12523856300115585, 'epoch': 111.11})

In [33]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): RWForCausalLM(
          (transformer): RWModel(
            (word_embeddings): Embedding(65024, 4544)
            (h): ModuleList(
              (0-31): 32 x DecoderLayer(
                (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
                (self_attention): Attention(
                  (maybe_rotary): RotaryEmbedding()
                  (query_key_value): Linear4bit(
                    in_features=4544, out_features=4672, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4544, out_features=64, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=64, out_featu

In [35]:
model.save_pretrained('trained_model')

In [43]:
from huggingface_hub import notebook_login
notebook_login()

In [44]:
model.push_to_hub("pgupta2023/falcon-7b-ft-prac")

CommitInfo(commit_url='https://huggingface.co/pgupta2023/falcon-7b-ft-prac/commit/1f58dd084740474afd94c31d888da0da81b33ca6', commit_message='Upload model', commit_description='', oid='1f58dd084740474afd94c31d888da0da81b33ca6', pr_url=None, pr_revision=None, pr_num=None)